In [945]:
%%pip install pandas --upgrade
%%pip install openpyxl --upgrade



In [946]:
# importing pandas and psycopg2

import pandas as pd
import psycopg2


In [947]:
# creating a function that connects to our original database to test our connection to postgres

def create_database():
    # connect to default database 
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=0609")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    # create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS solesociety")
    cur.execute("CREATE DATABASE solesociety")

    # close connection to default database
    conn.close()

    # connect to solesociety database
    conn = psycopg2.connect("host=localhost dbname=solesociety user=postgres password=0609")
    cur = conn.cursor()

    return cur, conn

In [948]:
# running this will create the solesociety database

cur, conn = create_database()

In [949]:
# function that drops tables

def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [950]:
# function that creates tables


def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [951]:
# EXTRACT 

# pulling each sheet of the solesociety data spreadsheet as its df so we can easily insert into tables later

style_df = pd.read_excel("/Users/rickycamilo/Desktop/PostgreSQL Bootcamp and SoleSociety ETL Project/SoleSociety ETL Migration SQL Project/SoleSociety Tables.xlsx", sheet_name="style")

brand_df = pd.read_excel("/Users/rickycamilo/Desktop/PostgreSQL Bootcamp and SoleSociety ETL Project/SoleSociety ETL Migration SQL Project/SoleSociety Tables.xlsx", sheet_name="brand")

customer_df = pd.read_excel("/Users/rickycamilo/Desktop/PostgreSQL Bootcamp and SoleSociety ETL Project/SoleSociety ETL Migration SQL Project/SoleSociety Tables.xlsx", sheet_name="customer")

country_df = pd.read_excel("/Users/rickycamilo/Desktop/PostgreSQL Bootcamp and SoleSociety ETL Project/SoleSociety ETL Migration SQL Project/SoleSociety Tables.xlsx", sheet_name="country")

address_df = pd.read_excel("/Users/rickycamilo/Desktop/PostgreSQL Bootcamp and SoleSociety ETL Project/SoleSociety ETL Migration SQL Project/SoleSociety Tables.xlsx", sheet_name="address")

store_df = pd.read_excel("/Users/rickycamilo/Desktop/PostgreSQL Bootcamp and SoleSociety ETL Project/SoleSociety ETL Migration SQL Project/SoleSociety Tables.xlsx", sheet_name="store")

staff_df = pd.read_excel("/Users/rickycamilo/Desktop/PostgreSQL Bootcamp and SoleSociety ETL Project/SoleSociety ETL Migration SQL Project/SoleSociety Tables.xlsx", sheet_name="staff")

payment_df = pd.read_excel("/Users/rickycamilo/Desktop/PostgreSQL Bootcamp and SoleSociety ETL Project/SoleSociety ETL Migration SQL Project/SoleSociety Tables.xlsx", sheet_name="payment")

sneaker_df = pd.read_excel("/Users/rickycamilo/Desktop/PostgreSQL Bootcamp and SoleSociety ETL Project/SoleSociety ETL Migration SQL Project/SoleSociety Tables.xlsx", sheet_name="sneaker")

In [952]:
# testing that we pulled our data into dfs successfully

customer_df.head()
staff_df.head()
customer_df.head()


,customer_id,store_id,address_id,customer_firstname,customer_lastname,email,create_date
0,1,1,1,Francoise,Rautenstrauch,francoise.rautenstrauch@rautenstrauch.com,2023-09-09
1,2,1,2,Kendra,Loud,kloud@gmail.com,2023-09-10
2,3,2,3,Lourdes,Bauswell,lourdes_bauswell@aol.com,2023-09-11
3,4,5,4,Hannah,Edmison,hannah@yahoo.com,2023-09-12
4,5,4,5,Tom,Loeza,tom.loeza@gmail.com,2023-09-13


In [953]:
# TRANSFORM

# concatinating the customer_firstname and customer_lastname columns in the staff table for later insertion into postgres database
# concatinating the staff_firstname and staff_lastname columns in the staff table for later insertion into postgres database
customer_df["customer_name"] = customer_df["customer_firstname"] + " " + customer_df["customer_lastname"]
staff_df["staff_name"] = staff_df["staff_firstname"] + " " + staff_df["staff_lastname"]




# selecting only the the columns I want to insert into postgres
customer_df = customer_df[["customer_id", "store_id", "address_id", "customer_name","email","create_date" ]]
staff_df = staff_df[["staff_id", "store_id", "staff_name","location","staff_email","username","password" ]]



In [954]:
# creating table and executing to postgres

style_table_create = ("""CREATE TABLE IF NOT EXISTS style(
style_id INT PRIMARY KEY,
style_name TEXT)                      
""")

cur.execute(style_table_create)
conn.commit()

In [955]:
brand_table_create = ("""CREATE TABLE IF NOT EXISTS brand(
brand_id INT PRIMARY KEY,
brand_name TEXT)                      
""")

cur.execute(brand_table_create)
conn.commit ()                     

In [956]:
sneaker_table_create = ("""CREATE TABLE IF NOT EXISTS sneaker(
sneaker_id BIGINT PRIMARY KEY ,
brand_id INT,
style_id INT,
store_id INT,
sneaker_name TEXT,
colorway TEXT,
release_year TEXT,
material TEXT,
price FLOAT,
size INT )                      
""")

cur.execute(sneaker_table_create)
conn.commit ()                           
                        

In [957]:
payment_table_create = ("""CREATE TABLE IF NOT EXISTS payment(
payment_id INT PRIMARY KEY,
customer_id INT,
staff_id INT,
payment_date DATE,
sneaker_id INT,
quantity INT,
amount INT)                      
""")

cur.execute(payment_table_create)
conn.commit()                           
             

In [958]:
staff_table_create = ("""CREATE TABLE IF NOT EXISTS staff(
staff_id INT PRIMARY KEY ,
store_id INT,
staff_name TEXT,
location TEXT,
staff_email TEXT,
username TEXT,
password TEXT )                      
""")

cur.execute(staff_table_create)
conn.commit ()                           
             

In [959]:
customer_table_create = ("""CREATE TABLE IF NOT EXISTS customer(
customer_id INT PRIMARY KEY,
store_id INT,
address_id INT,
customer_name TEXT,
email TEXT,
create_date DATE)                      
""")

cur.execute(customer_table_create)
conn.commit()                            
             

In [960]:
store_table_create = ("""CREATE TABLE IF NOT EXISTS store(
store_id INT PRIMARY KEY,
manager_staff_id INT,
store_location TEXT,
store_name TEXT )                      
""")

cur.execute(store_table_create)
conn.commit ()

In [961]:
address_table_create = ("""CREATE TABLE IF NOT EXISTS address(
address_id INT PRIMARY KEY ,
address TEXT,
country_id INT,
city TEXT,
state TEXT,
postal_code TEXT,
phone TEXT )                      
""")

cur.execute(address_table_create)
conn.commit()  

In [962]:
country_table_create = ("""CREATE TABLE IF NOT EXISTS country(
country_id INT PRIMARY KEY,
country TEXT                      
)""")

cur.execute(country_table_create)
conn.commit()

In [963]:
# inserting column names into solesociety database

style_df_table_insert = ("""INSERT INTO style(
style_id,
style_name)
VALUES (%s,%s)                                                                           
""")

In [964]:
store_df_table_insert = ("""INSERT INTO store(
store_id,
manager_staff_id,
store_location,
store_name)
VALUES(%s,%s,%s,%s)                         
""")

In [965]:
brand_df_table_insert = ("""INSERT INTO brand(
brand_id,
brand_name)
VALUES(%s,%s)                         
""")

In [966]:
sneaker_df_table_insert = ("""INSERT INTO sneaker(
sneaker_id,
brand_id,
style_id,
store_id,
sneaker_name,
colorway,
release_year,
material,
price,
size)
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
""")

In [967]:
payment_df_table_insert = ("""INSERT INTO payment(
payment_id,
customer_id,
staff_id,
payment_date,
sneaker_id,
quantity,
amount)
VALUES(%s,%s,%s,%s,%s,%s,%s)
""")

In [968]:
staff_df_table_insert = ("""INSERT INTO staff(
staff_id,
store_id,
staff_name,
location,
staff_email,
username,
password)
VALUES(%s,%s,%s,%s,%s,%s,%s)
""")

In [969]:
customer_df_table_insert = ("""INSERT INTO customer(
customer_id,
store_id,
address_id,
customer_name,
email,
create_date)
VALUES(%s,%s,%s,%s,%s,%s)
""")

In [970]:
country_df_table_insert = ("""INSERT INTO country(
country_id,
country)
VALUES(%s,%s)
""")

In [971]:
address_df_table_insert = ("""INSERT INTO address(
address_id,
address,
country_id,
city,
state,
postal_code,
phone)
VALUES(%s,%s,%s,%s,%s,%s,%s)
""")

In [972]:
# LOAD

# loop that iterates through every row of the mentioned dataframe. this will hydrate each table we created with the designated dataframe

for i, row in style_df.iterrows():
        cur.execute(style_df_table_insert, list(row))

conn.commit()


In [973]:
for i, row in brand_df.iterrows():
        cur.execute(brand_df_table_insert, list(row))

conn.commit()


In [974]:
for i, row in sneaker_df.iterrows():
        cur.execute(sneaker_df_table_insert, list(row))

conn.commit()

In [975]:
for i, row in payment_df.iterrows():
        cur.execute(payment_df_table_insert, list(row))

conn.commit()

In [976]:
for i, row in store_df.iterrows():
        cur.execute(store_df_table_insert, list(row))

conn.commit()

In [977]:
for i, row in staff_df.iterrows():
        cur.execute(staff_df_table_insert, list(row))

conn.commit()


In [978]:
for i, row in customer_df.iterrows():
        cur.execute(customer_df_table_insert, list(row))

conn.commit()


In [979]:
for i, row in address_df.iterrows():
        cur.execute(address_df_table_insert, list(row))

conn.commit()

In [980]:
for i, row in country_df.iterrows():
        cur.execute(country_df_table_insert, list(row))

conn.commit()